In [1]:
#imports 

import os 
import io 
import sys
from dotenv import load_dotenv
from openai import OpenAI
import anthropic 
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess


In [2]:
#load environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')


In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = 'gpt-4o'
CLAUDE_MODEL = 'claude-3-5-sonnet-20240620'
 

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):
    # Create a streaming completion using the prepared messages.
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    # Initialize an empty string to store the full reply.
    reply = ""
    # Process the streamed response.
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    # Save or process the complete reply.
    write_output(reply)

In [9]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
# Execute the provided Python code block as a string.
exec(pi)

Result: 3.141592658589
Execution Time: 21.239989 seconds


In [12]:
# Use OpenAI's GPT model to optimize and translate the Python code to faster C++.
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        int j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    using namespace std::chrono;
    
    int iterations = 100000000;
    int param1 = 4;
    int param2 = 1;
    
    auto start_time = high_resolution_clock::now();
    double result = calculate(iterations, param1, param2) * 4;
    auto end_time = high_resolution_clock::now();
    
    duration<double> exec_time = end_time - start_time;
    
    std::cout << "Result: " << std::setprecision(12) << std::fixed << result << std::endl;
    std::cout << "Execution Time: " << std::setprecision(6) << exec_time.count() << " seconds" << std::endl;
    
    return 0;
}
```


Compiling C++ and executing

In [13]:
# Compile C++ and run the executable
!g++ -O3 -std=c++17 -o optimized optimized.cpp
!optimized.exe


Result: 3.141592658590
Execution Time: 0.231503 seconds


In [14]:
# Use claude to optimize and translate the Python code to faster C++.
optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    #pragma omp parallel for reduction(-:result)
    for (long long i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i) * param1 - param2;
        result -= 1.0 / j;
        j = static_cast<double>(i) * param1 + param2;
        result += 1.0 / j;
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000LL, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    std::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;

    return 0;
}

In [15]:
# Compile C++ and run the executable
!g++ -O3 -std=c++17 -o optimized optimized.cpp
!optimized.exe


Result: 3.141592658590
Execution Time: 0.233033000000 seconds


In [16]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [17]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 58.200042 seconds


In [18]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <ctime>
#include <limits>
#include <cstdint>

// Linear Congruential Generator
class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1U << 31)
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t value, a, c, m;
};

// Calculate the maximum subarray sum using Kadane's algorithm
int max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    int max_sum = std::numeric_limits<int>::min(), current_sum = 0;
    for (int i = 0; i < n; ++i) {
        current_sum += random_numbers[i];
        if (current_sum > max_sum)
            max_sum = current_sum;
        if (current_sum < 0)
            current_sum = 0

In [19]:
!g++ -O3 -std=c++17 -o optimized optimized.cpp
!optimized.exe

Total Maximum Subarray Sum (20 runs): 11720
Execution Time: 0.000000 seconds


In [20]:
optimize_claude(python_hard)

#include <iostream>
#include <vector>
#include <chrono>
#include <limits>
#include <iomanip>

using namespace std;
using namespace chrono;

class LCG {
private:
    uint64_t value;
    const uint64_t a = 1664525;
    const uint64_t c = 1013904223;
    const uint64_t m = (1ULL << 32);

public:
    LCG(uint64_t seed) : value(seed) {}

    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

int64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    vector<int64_t> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = numeric_limits<int64_t>::min();
    int64_t current_sum = 0;
    int64_t min_sum = 0;

    for (int i = 0; i < n; ++i) {
        current_sum += random_numbers[i];
        max_sum = max(max_sum, current_sum - min_sum);
        min_sum = min(min_sum, current_sum);
    }

    return max_sum;
}

int64_t t

In [21]:
!g++ -O3 -std=c++17 -o optimized optimized.cpp
!optimized.exe

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001031 seconds


In [22]:
def stream_gpt(python):
    # Initiate a streaming completion request to the OpenAI API.
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,  # Specify the model to use (e.g., "gpt-4o").
        messages=messages_for(python),  # Prepare the input messages based on the Python code.
        stream=True  # Enable streaming for incremental response generation.
    )
    
    reply = ""  # Initialize an empty string to store the response incrementally.

    # Process the streamed response.
    for chunk in stream:
        # Extract the content from the current chunk or default to an empty string.
        fragment = chunk.choices[0].delta.content or ""
        
        # Append the fragment to the accumulated reply.
        reply += fragment

        # Yield the current reply, removing specific markdown formatting.
        yield reply.replace('```cpp\n', '').replace('```', '')


In [23]:
def stream_claude(python):
    # Initiate a streaming request to Claude's API with the specified parameters.
    result = claude.messages.stream(
        model=CLAUDE_MODEL,  # Specify the Claude model to use.
        max_tokens=2000,  # Limit the response to a maximum of 2000 tokens.
        system=system_message,  # Provide a system-level instruction or context.
        messages=[{"role": "user", "content": user_prompt_for(python)}],  # Prepare the user message with the Python code.
    )
    
    reply = ""  # Initialize an empty string to store the response incrementally.

    # Process the streamed response.
    with result as stream:  # Open the result as a stream.
        for text in stream.text_stream:  # Iterate over each streamed text chunk.
            reply += text  # Append the current text chunk to the accumulated reply.
            # Yield the current reply, removing specific markdown formatting.
            yield reply.replace('```cpp\n', '').replace('```', '')


In [24]:
def optimize(python, model):
    # Determine which model to use and call the corresponding streaming function.
    if model == "GPT":
        result = stream_gpt(python)  # Call the GPT streaming function if the model is "GPT".
    elif model == "Claude":
        result = stream_claude(python)  # Call the Claude streaming function if the model is "Claude".
    else:
        # Raise an error if the model is not recognized.
        raise ValueError("Unknown model")
    
    # Process and yield the streamed results incrementally.
    for stream_so_far in result:
        yield stream_so_far


In [25]:
with gr.Blocks() as ui:  # Create a Gradio interface block.
    with gr.Row():  # Define the first row of the UI.
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)  # Input for Python code.
        cpp = gr.Textbox(label="C++ code:", lines=10)  # Output for converted C++ code.
    
    with gr.Row():  # Define the second row of the UI.
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")  # Dropdown to select the model.
        convert = gr.Button("Convert code")  # Button to trigger code conversion.

    # Link the 'convert' button to the 'optimize' function.
    # Inputs: Python code and selected model.
    # Outputs: Converted C++ code.
    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)  # Launch the Gradio UI in the browser.

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [26]:
def execute_python(code):
    try:
        # Redirect standard output to a StringIO object to capture print statements.
        output = io.StringIO()  # Create a StringIO object to store the output.
        sys.stdout = output  # Redirect standard output to the StringIO object.
        
        # Execute the provided Python code string.
        exec(code)
    finally:
        # Restore standard output to its original state to prevent side effects.
        sys.stdout = sys.__stdout__
    
    # Return the captured output as a string.
    return output.getvalue()



In [27]:
def execute_cpp(code):
    # Save the provided C++ code to a file for compilation and execution.
    write_output(code)
    
    try:
        # Compile the C++ code with optimization flags for performance.
        compile_cmd = [
            "g++", "-Ofast", "-std=c++17",  # Optimization and C++17 standard.
            "-march=native", "-mtune=native",  # Optimize for the native CPU architecture.
            "-o", "optimized", "optimized.cpp"  # Output executable named "optimized".
        ]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)  # Compile the code.

        # Run the compiled executable.
        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)  # Execute the compiled program.

        # Return the standard output of the program execution.
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        # Return the error message if the compilation or execution fails.
        return f"An error occurred:\n{e.stderr}"


In [28]:
css = """
.python {background-color: #306998;}  # Custom background for Python output.
.cpp {background-color: #050;}  # Custom background for C++ output.
"""

In [29]:
with gr.Blocks(css=css) as ui:  # Define the Gradio UI with custom CSS styling.
    gr.Markdown("## Convert code from Python to C++")  # Add a title to the UI.

    with gr.Row():  # Row for input textboxes.
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)  # Input for Python code.
        cpp = gr.Textbox(label="C++ code:", lines=10)  # Output for converted C++ code.

    with gr.Row():  # Row for model selection dropdown.
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")  # Dropdown to select the model.

    with gr.Row():  # Row for conversion button.
        convert = gr.Button("Convert code")  # Button to trigger code conversion.

    with gr.Row():  # Row for execution buttons.
        python_run = gr.Button("Run Python")  # Button to execute the Python code.
        cpp_run = gr.Button("Run C++")  # Button to execute the C++ code.

    with gr.Row():  # Row for output display areas.
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])  # Display Python execution result.
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])  # Display C++ execution result.

    # Link the conversion button to the optimize function.
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    # Link the Python execution button to the execute_python function.
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    # Link the C++ execution button to the execute_cpp function.
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)  # Launch the Gradio interface in the browser.


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
